In [36]:
import pyalex
import numpy
import networkx

In [2]:
authors_ids = ['A5079492698','A5030061016','A5011937105','A5017806956','A5088693574','A5068784622','A5033397019','A5042762772','A5056479011','A5066452403','A5023920432','A5014023844','A5074410797','A5053494534','A5010023742','A5048975755','A5018814072','A5068040320','A5001900540','A5055438571','A5027230106','A5012497783','A5046094094']

In [13]:
pager = pyalex.Works().filter(authorships={"author": {"id": "|".join(authors_ids)}}).sort(cited_by_count="desc").paginate(per_page=200)
works = []
for page in pager:
    for work in page:
        if work is not None:
            works.append(work)

In [31]:
network_authors = []
for work in works:
    for item in work['authorships']:
        author_id = item['author']['id'][21:]
        author_name = item['author']['display_name']
        author = (author_id, author_name)
        if author not in network_authors:
            network_authors.append(author)

In [38]:
matrix = numpy.zeros((len(network_authors), len(network_authors)), dtype=int)
for work in works:
    for author1 in work['authorships']:
        a = network_authors.index((author1['author']['id'][21:], author1['author']['display_name']))
        for author2 in work['authorships']:
            b = network_authors.index((author2['author']['id'][21:], author2['author']['display_name']))
            matrix[a][b] += 1

In [40]:
graph = networkx.from_numpy_array(matrix, parallel_edges=False)
graph.remove_edges_from(networkx.selfloop_edges(graph))
attributes = {i: {"openalex_id": network_authors[i][0], "name": network_authors[i][1]}  for i in range(len(network_authors))}
networkx.set_node_attributes(graph, attributes)

In [42]:
networkx.density(graph)

0.011452563873503662

In [43]:
networkx.average_degree_connectivity(graph)

{124: 7.645161290322581,
 31: 18.0,
 20: 17.283333333333335,
 44: 10.136363636363637,
 1: 48.41860465116279,
 6: 23.22685185185185,
 51: 9.509803921568627,
 15: 18.177777777777777,
 5: 26.836363636363636,
 88: 10.090909090909092,
 3: 28.159420289855074,
 4: 26.945,
 13: 12.615384615384615,
 2: 31.62087912087912,
 8: 21.445652173913043,
 12: 23.61111111111111,
 16: 19.4375,
 9: 17.0,
 81: 7.567901234567901,
 11: 17.766233766233768,
 24: 12.729166666666666,
 10: 21.377777777777776,
 45: 9.9,
 61: 7.426229508196721,
 14: 25.685714285714287,
 25: 6.76,
 66: 12.272727272727273,
 18: 14.777777777777779,
 7: 16.62,
 21: 1.7619047619047619,
 29: 13.172413793103448,
 19: 18.652631578947368,
 26: 15.615384615384615,
 22: 17.90909090909091}

In [47]:
maxDegree = max(list(graph.degree()), key=lambda x: x[1])[0]
graph.nodes[maxDegree]

{'openalex_id': 'A5068784622', 'name': 'David Nadler Prata'}

In [50]:
[maxWeight1, maxWeight2, value] = max(list(graph.edges(data=True)), key=lambda x: x[2]['weight'])
print(graph.nodes[maxWeight1])
print(graph.nodes[maxWeight2])

{'openalex_id': 'A5068784622', 'name': 'David Nadler Prata'}
{'openalex_id': 'A5053494534', 'name': 'Marcelo Lisboa Rocha'}


In [57]:
qtdWeightOne = len([value for value in list(graph.edges(data=True)) if value[2]['weight'] == 13])
qtdWeightOne

0